<a href="https://colab.research.google.com/github/Pratikcsc/ContextBasedCongestion/blob/master/ecoli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# GET X
import pandas as pd

data_X = pd.read_csv('tweet_vector.csv')

data_X.shape


(8329, 100)

In [0]:
# Get Y

data_Y = pd.read_csv("ground_ruth_classLabel.csv")

data_Y.shape

(8329, 1)

In [0]:
# encoding column 'c7'
import numpy as np

def allLabels(data_Y):
  labels = []
  #count = np.zeros(10)
  for i in data_Y.iloc[:,0]:
    if i not in labels:
      labels.append(i)
  return labels

In [0]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

def GaussianNBModel(X_train, X_test, y_train, y_test):
  model = GaussianNB()
  
  model.fit(X_train, y_train)
  #print("Gaussian NB Regression Score = {}".format(model.score(X_train,y_train)))
  
  prediction = model.predict(X_test)
  return (y_test, prediction)
  
  
def LogisticRegressionModel(X_train, X_test, y_train, y_test):
  model = LogisticRegression()
  
  model.fit(X_train,y_train)
  
  #print("Logistic Regression Score = {}".format(model.score(X_train,y_train)))
  
  prediction = model.predict(X_test)
  return (y_test, prediction)
  
  
def SVCModel(X_train, X_test, y_train, y_test):
  model = SVC()
  model.fit(X_train,y_train)
  
  #print("SVC Score = {}".format(model.score(X_train,y_train)))
  
  prediction = model.predict(X_test)
  return (y_test, prediction)

def SGDClassifierModel(X_train, X_test, y_train, y_test):
  model = SGDClassifier(loss="hinge", shuffle = True)
  #sgdModel = SGDClassifier(loss="hinge")
  model.fit(X_train,y_train)
  
  score = model.score(X_train, y_train)
  
  #print("SGD Score = {}".format(score))
  
  prediction = model.predict(X_test)
  
  return (y_test, prediction)
  


In [0]:
# MAIN EXECUTABLE CODE SNIPPET
import warnings

warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = train_test_split(data_X, data_Y)

print("Length of training X = {} and training Y = {}\nLength of test X = {} and test Y = {}".format(len(X_train), len(y_train), len(X_test), len(y_test)))

#labels = allLabels()


Length of training X = 6246 and training Y = 6246
Length of test X = 2083 and test Y = 2083


In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

def getConfusionMatrix(resTuple, labels, model):
  matrix = confusion_matrix(y_true = resTuple[0], y_pred = resTuple[1], labels = labels)
  return (matrix, model)

def getF1Score(resTuple, labels, model):
  matrix = f1_score(y_true = resTuple[0], y_pred = resTuple[1], labels = labels, average = 'micro')
  return (matrix, model)


In [0]:
allLabels(data_Y)

[3, 1, -1, 0, 2, 4]

In [0]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
#X = [['Male', 1], ['Female', 3], ['Female', 2]]
_y_train = enc.fit_transform(y_train)
_y_test = enc.fit_transform(y_test)

In [0]:
labels = allLabels(data_Y)
modelList = ['SVC', 'LR', 'SGD', 'GNB']
resTuples = [SVCModel(X_train, X_test, y_train, y_test),\
             LogisticRegressionModel(X_train, X_test, y_train, y_test),\
             SGDClassifierModel(X_train, X_test, y_train, y_test),\
             GaussianNBModel(X_train, X_test, y_train, y_test)
            ]

f = open("/content/result.csv", "w")

for i in range(len(resTuples)):
  res1 = getConfusionMatrix(resTuple = resTuples[i], labels = labels, model = modelList[i])
  res2 = getF1Score(resTuple = resTuples[i], labels = labels, model = modelList[i])
  print("{}, {}, {}\n".format(res1[0], res2[0], res1[1]))
  #f.write("{}, {}, {}\n".format(res1[0], res2[0], res1[1]))

[[390   5   0   0   1   0]
 [ 11 449  17   3   0   0]
 [  2  14 246   3   0   2]
 [  0   5   4 512   0   1]
 [  0   0   0   3 350   0]
 [  1   2  11   0   3  48]], 0.957753240518483, SVC

[[390   5   0   0   1   0]
 [ 10 455  11   4   0   0]
 [  3  14 234   5   2   9]
 [  0   2   1 516   2   1]
 [  0   2   0   3 348   0]
 [  0   0  11   0   1  53]], 0.9582333173307729, LR

[[393   2   0   0   1   0]
 [ 14 451  14   0   1   0]
 [  4  11 248   2   0   2]
 [  0   4   5 507   5   1]
 [  0   3   0   1 349   0]
 [  1   4  14   0   3  43]], 0.9558329332693231, SGD

[[360  18   3   1   6   8]
 [ 10 340  52  28  19  31]
 [  0  61 158  13   7  28]
 [  0  47  29 399  39   8]
 [  0  37   9  17 289   1]
 [  0   3   3  12   3  44]], 0.7633221315410466, GNB



In [0]:
from tensorflow import keras 
from tensorflow.keras import layers
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import warnings

warnings.filterwarnings('ignore')


model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=100))
model.add(Dense(units=125, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=6, activation='sigmoid'))
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(X_train, _y_train, epochs=10, batch_size=32)
loss_and_metrics = model.evaluate(X_test, _y_test, batch_size=128)

loss_and_metrics


Epoch 1/10
6246/6246 [==============================] - 1s 106us/step - loss: 1.3854 - acc: 0.4821
Epoch 2/10
6246/6246 [==============================] - 0s 62us/step - loss: 0.7693 - acc: 0.7565
Epoch 3/10
6246/6246 [==============================] - 0s 61us/step - loss: 0.5218 - acc: 0.8325
Epoch 4/10
6246/6246 [==============================] - 0s 62us/step - loss: 0.3922 - acc: 0.8753
Epoch 5/10
6246/6246 [==============================] - 0s 61us/step - loss: 0.3469 - acc: 0.8842
Epoch 6/10
6246/6246 [==============================] - 0s 60us/step - loss: 0.3128 - acc: 0.8995
Epoch 7/10
6246/6246 [==============================] - 0s 60us/step - loss: 0.2673 - acc: 0.9153
Epoch 8/10
6246/6246 [==============================] - 0s 63us/step - loss: 0.2667 - acc: 0.9209
Epoch 9/10
6246/6246 [==============================] - 0s 60us/step - loss: 0.2302 - acc: 0.9323
Epoch 10/10
2083/2083 [==============================] - 0s 61us/step


[0.20490543138677547, 0.9371099376758588]